In [9]:
# Got from https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/
#%env CUDA_VISIBLE_DEVICES=2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [10]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

W0819 07:28:11.408615 14036 hook.py:98] Torch was already hooked... skipping hooking process


In [11]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 2     ## Changed by me from 10 to 2
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True  ## Changed by me from false to true
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [12]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [14]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [15]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")


Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.273475
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.216173
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.139428
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.053070
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896594
Train Epoch: 1 [4480/60032 (7%)]	Loss: 1.917290
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.655101
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440288
Train Epoch: 1 [6400/60032 (11%)]	Loss: 1.231442
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.983658
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.867165
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.890829
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.862046
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654450
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.586885
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.692936
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593086
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.532170
Train Epoch: 1 [12800/60032 (

Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.101714
Train Epoch: 2 [45440/60032 (76%)]	Loss: 0.115084
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.085339
Train Epoch: 2 [46720/60032 (78%)]	Loss: 0.073035
Train Epoch: 2 [47360/60032 (79%)]	Loss: 0.195450
Train Epoch: 2 [48000/60032 (80%)]	Loss: 0.101181
Train Epoch: 2 [48640/60032 (81%)]	Loss: 0.082996
Train Epoch: 2 [49280/60032 (82%)]	Loss: 0.095236
Train Epoch: 2 [49920/60032 (83%)]	Loss: 0.155644
Train Epoch: 2 [50560/60032 (84%)]	Loss: 0.106655
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.061939
Train Epoch: 2 [51840/60032 (86%)]	Loss: 0.031716
Train Epoch: 2 [52480/60032 (87%)]	Loss: 0.169198
Train Epoch: 2 [53120/60032 (88%)]	Loss: 0.070003
Train Epoch: 2 [53760/60032 (90%)]	Loss: 0.072619
Train Epoch: 2 [54400/60032 (91%)]	Loss: 0.050008
Train Epoch: 2 [55040/60032 (92%)]	Loss: 0.098170
Train Epoch: 2 [55680/60032 (93%)]	Loss: 0.114193
Train Epoch: 2 [56320/60032 (94%)]	Loss: 0.092082
Train Epoch: 2 [56960/60032 (95%)]	Loss: 0.041913
